In [1]:
import PIL
import torch
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from Dataset.AerialDataset import AerialDataset
from tasks.trainer import Trainer

- Generar imagenes bicubicas
- Construir Dataset
- Construir Dataloader
- SR3
- SRdiff
- SR3+

## 64 -> 256


#### Generación de imagenes bicubicas

In [ ]:
import os
import numpy as np
from scipy.interpolate import interpn
from PIL import Image
from tqdm import tqdm

##TODO Mover esto a su propio modulo donde poder usarlo para preparar los datos a posteriori.
## TODO cambiar metodo para q vaya más rapido.


def bicubic_interpolation(image, objective_dim): # De momento lo implementare para 1 sola foto a la vez
    #Calculo nuevas dimensiones
    height, width = image.shape[0] , image.shape[1]
    new_width, new_height = objective_dim[0], objective_dim[1]
    new_image = np.zeros((new_height, new_width, image.shape[2]))
    
     # Generar cuadrículas para las coordenadas X e Y de la imagen original y la interpolada
    x = np.linspace(0, width - 1, width)
    y = np.linspace(0, height - 1, height)
    new_x = np.linspace(0, width - 1, new_width)
    new_y = np.linspace(0, height - 1, new_height)
    new_image = interpn((y, x), image, (new_y[:,None], new_x), method='cubic', bounds_error=False, fill_value=0)
    return new_image

dataset_dir = 'E:\\TFG\\air_dataset\\64'
for image_file in tqdm(os.listdir(dataset_dir)):
    image = Image.open(os.path.join(dataset_dir, image_file))
    image = np.array(image)
    interpolated = bicubic_interpolation(image, (256, 256))
    interpolated_image = Image.fromarray(interpolated.astype(np.uint8))
    interpolated_image.save(f'E:\\TFG\\air_dataset\\sr\\64_256\\{image_file}')
    

## Entrenamiento

In [2]:
lr_size = 64
hr_size = 256
batch_size = 64
epochs = 5
learning_rate = 0.001
dataset_dir = 'E:\\TFG\\air_dataset'

logs_dir = 'E:\\TFG\\logs'

In [4]:
dataset = AerialDataset(dataset_dir, lr_size, hr_size)
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

trainer = Trainer(logs_dir=logs_dir)
for epoch in range(epochs):
    


torch.Size([64, 3, 256, 256])
torch.Size([64, 3, 256, 256])
torch.Size([64, 3, 256, 256])


KeyboardInterrupt: 